<a href="https://colab.research.google.com/github/BillieOdelle/deep-learning-challenge/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [56]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [57]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=['EIN', 'NAME'])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [58]:
# Determine the number of unique values in each column.
unique_value = df.nunique()
unique_value

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [59]:
# Look at APPLICATION_TYPE value counts for binning
application_types = df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [60]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 50
application_types_to_replace = application_types[application_types <= cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [61]:
# Look at CLASSIFICATION value counts for binning
classification_types = df['CLASSIFICATION'].value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [62]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_types = classification_types[classification_types > 1]
print(classification_types)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [63]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1500
classifications_to_replace = application_types[application_types <= cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [64]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT',	'IS_SUCCESSFUL',]
df_encoded = pd.get_dummies(encoded_columns)
df_encoded

,APPLICATION_TYPE,ASK_AMT,CLASSIFICATION,INCOME_AMT,IS_SUCCESSFUL,ORGANIZATION
0,1,0,0,0,0,0
1,0,0,1,0,0,0
2,0,0,0,0,0,1
3,0,0,0,1,0,0
4,0,1,0,0,0,0
5,0,0,0,0,1,0


In [65]:
# Separate the data into labels and features
labels = ['APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT',	'IS_SUCCESSFUL',]
features = df_encoded

In [66]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop(columns='IS_SUCCESSFUL')
y = df_encoded['IS_SUCCESSFUL']


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(128, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(64, activation='relu'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(32, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 256)               1536      
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dense_19 (Dense)            (None, 1)                 33        
                                                                 
Total params: 44,801
Trainable params: 44,801
Non-trainable params: 0
_________________________________________________________________


In [73]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
# Train the model
epochs = 10
batch_size = 50
nn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.5504 - accuracy: 0.6667 - val_loss: 0.5123 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 36ms/step - loss: 0.5330 - accuracy: 0.6667 - val_loss: 0.4925 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 39ms/step - loss: 0.5165 - accuracy: 0.6667 - val_loss: 0.4709 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 57ms/step - loss: 0.5003 - accuracy: 0.6667 - val_loss: 0.4494 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 38ms/step - loss: 0.4842 - accuracy: 0.6667 - val_loss: 0.4283 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 55ms/step - loss: 0.4683 - accuracy: 0.6667 - val_loss: 0.4079 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 37ms/step - loss: 0.4524 - accuracy: 0.6667 - val_loss: 0.3883 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=============

In [75]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.3892 - accuracy: 1.0000 - 146ms/epoch - 146ms/step
Loss: 0.3892219364643097, Accuracy: 1.0


In [72]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')
loaded_model = tf.keras.models.load_model('AlphabetSoupCharity.h5')